# Type Instability

So far, as we've written Julia code, we've come to understand that to get good performance, we need to: 
1. Make sure our operations don't take "too much" memory
2. Make sure there's no "unboxing" going on
3. In other words, exposing as much type information to the compiler as possible. 

In this notebook we're going to expand on Pt 3. How exactly do we write code that plays well with the compiler?

In [ ]:
sqrt(4)

In [ ]:
sqrt(-2)

This looks strange, doesn't it? Why does Julia behave this way?

In [ ]:
sqrt(-1 + 0im)

To understand, let's make our own `sqrt` function. 

In [ ]:
mysqrt(x::Complex) = sqrt(x)
mysqrt(x::Real) = x < 0 ? sqrt(complex(x)) : sqrt(x)

In [ ]:
mysqrt(2)

In [ ]:
mysqrt(-2)

In [ ]:
mysqrt(-2+0im)

This works just like we want it to. Right?

In [ ]:
a = randn(10^7);

In [ ]:
@time c1 = mysqrt.(a)

In [ ]:
@time c2 = sqrt.(Complex.(a))

In [ ]:
c1 == c2

Why is this happening? To understand why, let us look at the `@code_warntype` inspection tool.

In [ ]:
slowfun(x) = mysqrt(x) + 1
@code_warntype slowfun(2)

**A familiar example**: our sum function

In [ ]:
function mysum_slow(a)
    s = 0
    for x in a
        s += x
    end
    return s
end

In [ ]:
mysum_slow([3,4,5])

In [ ]:
mysum_slow([3.3,4,5])

In [ ]:
@code_warntype(mysum_slow([3.3,4,5]))